In [3]:
from nba_api.stats.endpoints import playercareerstats, playergamelog, playergamelogs
from nba_api.stats.static import players
import pandas as pd
from io import BytesIO

In [7]:
def get_player_id_from_name(player_name):
    all_players = players.get_players()
    for player in all_players:
        if player['full_name'].lower() == player_name.lower():
            return player['id']
    return None

In [32]:
player_name = "Luka Doncic"
player_id = get_player_id_from_name(player_name)
url = f'https://cdn.nba.com/headshots/nba/latest/1040x760/{player_id}.png'

response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = await deeppyer.deepfry(img, flares=False)
img.save('./bar.jpg')

In [28]:
# Set player name and retrieve player ID
player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)

# Retrieve player career statistics
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_stats = career.get_data_frames()[0]

# Define column lists
totals_cols = ['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA',
            'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
            'OREB', 'DREB', 'PF']
per_game_cols = ['MPG', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'TPG', 'FGM', 'FGA',
                'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                'ORPG', 'DRPG', 'PF']
percentage_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT']

# Career Totals
career_totals_per_season = career_stats[['SEASON_ID', 'GP'] + totals_cols].copy()
career_totals_per_season['FULL_NAME'] = player_name

# Sum career totals and convert to int
career_totals = {'FULL_NAME': player_name, 'GP': int(career_totals_per_season['GP'].sum())}
career_totals.update(career_totals_per_season[totals_cols].sum().astype(int).to_dict())

# Convert non-integer columns to int except for 'FULL_NAME', 'SEASON_ID', and percentage columns
for col in career_totals_per_season.columns:
    if col not in ['FULL_NAME', 'SEASON_ID'] + percentage_stats:
        career_totals_per_season[col] = career_totals_per_season[col].astype(int)

# Career Averages
career_avg_per_season = pd.DataFrame(columns=['FULL_NAME', 'SEASON_ID', 'GP'] + per_game_cols)

# Calculate per game averages
for stat, per_game_stat in zip(totals_cols, per_game_cols):
    career_avg_per_season[per_game_stat] = round(career_stats[stat] / career_stats['GP'], 1)

career_avg_per_season['SEASON_ID'] = career_stats['SEASON_ID']
career_avg_per_season['GP'] = career_stats['GP']
career_avg_per_season['FULL_NAME'] = player_name

# Calculate career averages
career_averages = {'FULL_NAME': player_name, 'GP': int(career_avg_per_season['GP'].sum())}
career_averages.update(round(career_avg_per_season[per_game_cols].mean(), 1).to_dict())

# Convert percentage Stats
for stat in percentage_stats:
    career_avg_per_season[stat] = round(career_stats[stat] * 100, 1)
    career_averages[stat] = round(career_avg_per_season[stat].mean(), 1)
    career_totals_per_season[stat] = round(career_totals_per_season[stat] * 100, 1)
    career_totals[stat] = round(career_avg_per_season[stat].mean(), 1)

# Convert DataFrames to dictionaries
career_totals_per_season_json = career_totals_per_season.to_dict(orient='records')
career_avg_per_season_json = career_avg_per_season.to_dict(orient='records')

career_totals

{'FULL_NAME': 'LeBron James',
 'GP': 1492,
 'MIN': 56596,
 'PTS': 40474,
 'REB': 11185,
 'AST': 11009,
 'STL': 2275,
 'BLK': 1111,
 'TOV': 5211,
 'FGM': 14837,
 'FGA': 29313,
 'FG_PCT': 50.8,
 'FG3M': 2410,
 'FG3A': 6926,
 'FG3_PCT': 34.8,
 'FTM': 8390,
 'FTA': 11404,
 'FT_PCT': 73.4,
 'OREB': 1727,
 'DREB': 9458,
 'PF': 2682}

In [16]:
5.538095238095238/7.55238095238095

0.7332912988650695

In [56]:
def get_player_image(player_name):
    """
    Gets NBA player headshot using player ID.

    Args:
        - player_name (str): Full name of current or former NBA player.

    Returns:
        - URL link to the headshot of NBA player.
    """

    player_id = get_player_id_from_name(player_name=player_name)
    url = f'https://cdn.nba.com/headshots/nba/latest/1040x760/{player_id}.png'

    return url

get_player_image("Jason Kidd")

'https://cdn.nba.com/headshots/nba/latest/1040x760/467.png'